# 🧠 Multi-Agent AI System for Financial Transaction Monitoring

---

## 📄 Project Description

This notebook implements a **modular, multilingual AI system** for analyzing financial transaction messages. It uses a **multi-agent architecture**, where each agent handles a specific function — from natural language translation to fraud detection and database storage.

---

## 🔧 Key Features

### ✅ Multi-Agent Design

Each component is an autonomous agent:

- **🗣️ LanguageAgent**  
  Translates user input from any language into English using Google Translate.

- **🔍 ParsingAgent**  
  Extracts the amount, currency, and recipient from free-form messages.

- **🛡️ FraudDetectionAgent**  
  Applies a trained ML model to flag potentially suspicious transactions.

- **🗄️ DatabaseAgent**  
  Stores all transactions into a real-time SQLite database for audit and analysis.

- **📢 ReportingAgent**  
  Generates a natural-language report summarizing the transaction and its risk status.

---

## 🌐 Language Support

- Accepts messages in _any language_ (e.g., English, Persian).
- Translates input to English for uniform processing.
- Stores the original message and translated version in the database.

---

## 🧪 Example Use Cases

- `"Transfer 5000 dollars to Jack"` → ✅ Safe Transaction  
- `"ارسال ۸۰۰۰ تومان به علی"` → ✅ Processed via translation  
- `"Send 9000 to Unknown"` → ❗ Flagged as suspicious

---

## 🧰 Technologies Used

- **Python 3**
- **Gradio** for interactive interface
- **Google Translate API**
- **SQLite** for persistent storage
- **Scikit-learn** for fraud detection
- **Pandas** for data processing

---

## 🧠 Designed For:

- Financial AI research
- Fraud detection proof-of-concepts
- Chatbot and agent-based systems
- Educational or prototype banking apps

---



In [1]:
pip install gradio pandas scikit-learn googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of gradio to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of gradio to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:

import re
import sqlite3
import pandas as pd
from datetime import datetime
from googletrans import Translator
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import gradio as gr


## 🗣️ 1. LanguageAgent – Translation & Normalization

In [4]:
class LanguageAgent:
    def __init__(self):
        self.translator = Translator()
    
    def translate_to_english(self, message):
        translated = self.translator.translate(message, dest="en")
        return translated.text


## 🔍 2. ParsingAgent – Transaction Extraction

In [5]:
class ParsingAgent:
    def parse(self, message):
        pattern = r"(?i)(?:transfer|send|pay)?\s*(\d+)\s*(dollars|usd|euro|eur|toman|rial)?\s*(?:to)?\s*([A-Za-z]+)"
        match = re.search(pattern, message)

        if match:
            return {
                "amount": int(match.group(1)),
                "currency": match.group(2) or "USD",
                "recipient": match.group(3)
            }
        return None


## 🛡️ 3. FraudDetectionAgent – Machine Learning Model

In [6]:
class FraudDetectionAgent:
    def __init__(self):
        self.model = RandomForestClassifier()
        self.encoder = LabelEncoder()
        self._train_model()
    
    def _train_model(self):
        df = pd.DataFrame({
            "amount": [100, 5000, 200, 9000, 150, 7500],
            "recipient": ["Ali", "Reza", "Mina", "Mahdi", "Ali", "Unknown"],
            "is_fraud": [0, 1, 0, 1, 0, 1]
        })
        df["recipient_encoded"] = self.encoder.fit_transform(df["recipient"])
        self.model.fit(df[["amount", "recipient_encoded"]], df["is_fraud"])

    def is_fraudulent(self, transaction):
        try:
            encoded = self.encoder.transform([transaction["recipient"]])[0]
        except:
            encoded = -1
        return self.model.predict([[transaction["amount"], encoded]])[0] == 1


## 🗄️ 4. DatabaseAgent – Store Transaction

In [7]:
class DatabaseAgent:
    def __init__(self, db_name="transactions.db"):
        self.conn = sqlite3.connect(db_name, check_same_thread=False)
        self._create_table()

    def _create_table(self):
        self.conn.execute("""
            CREATE TABLE IF NOT EXISTS transactions (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                message TEXT,
                recipient TEXT,
                amount INTEGER,
                currency TEXT,
                is_fraud INTEGER,
                created_at TIMESTAMP
            )
        """)
        self.conn.commit()

    def store(self, message, transaction, is_fraud):
        self.conn.execute("""
            INSERT INTO transactions (message, recipient, amount, currency, is_fraud, created_at)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (
            message,
            transaction["recipient"],
            transaction["amount"],
            transaction["currency"],
            int(is_fraud),
            datetime.now()
        ))
        self.conn.commit()


## 📢 5. ReportingAgent – Generate Human Response

In [8]:
class ReportingAgent:
    def generate(self, transaction, is_fraud):
        status = "❗ Suspicious" if is_fraud else "✅ Safe"
        return (
            f"💳 Amount: {transaction['amount']} {transaction['currency']}\n"
            f"👤 Recipient: {transaction['recipient']}\n"
            f"🔍 Status: {status}"
        )


## 🤖 Main AI Agent – Coordinator

In [9]:
# Instantiate agents
language_agent = LanguageAgent()
parsing_agent = ParsingAgent()
fraud_agent = FraudDetectionAgent()
database_agent = DatabaseAgent()
reporting_agent = ReportingAgent()

def ai_transaction_agent(message):
    try:
        translated = language_agent.translate_to_english(message)
        transaction = parsing_agent.parse(translated)

        if not transaction:
            return "❌ Unable to parse the transaction. Please rephrase your message."

        is_fraud = fraud_agent.is_fraudulent(transaction)
        database_agent.store(message, transaction, is_fraud)
        report = reporting_agent.generate(transaction, is_fraud)

        return report

    except Exception as e:
        return f"❌ Error: {str(e)}"


## 🌐 Gradio UI

In [10]:
gr.Interface(
    fn=ai_transaction_agent,
    inputs=gr.Textbox(label="Enter your transaction message (any language)"),
    outputs=gr.Textbox(label="AI Report"),
    title="🤖 Multi-Agent AI Transaction Monitor",
    description="Type a message like 'Transfer 5000 dollars to Reza' or 'ارسال ۸۰۰۰ تومان به علی'."
).launch(share=True)


Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 4.19.1, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://39acacf6e5ef1d0e44.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
